In [1]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
random.seed(2)
qpa = qparray.QpixAsicArray(4, 4, debug=0.0)

In [2]:
timeNow = 1.
timeEnd = 4.0
tickNow = 50e6
deltaT = 1.0
deltaTick = 50e6
timeEpsilon = 1e-6
eventTimes = []
hitsPerEvent, eventTimes = [], []
procQueue = ProcQueue()
    
while(timeNow < timeEnd):

    items = 0
    topProc = []
    midProc = []
    botProc = []
    lenRecv = []
    asicRow = []
    asicCol = []

    for asic in qpa:
        newProcessItems = asic.Process(timeNow - timeEpsilon)
        if newProcessItems:
            print("WARNING: ASIC had things left to do at next maor time step")

    procQueue.AddQueueItem(qpa[0][0], 3, PixelHit(tickNow, [], None, None), timeNow)

    while(procQueue.Length() > 0):

        nextItem = procQueue.PopQueue()
        items += 1

        topProc.append(qpa.ProcessArray(procQueue, nextItem.inTime))

        newProcessItems = nextItem.asic.ReceiveData(nextItem)
        if newProcessItems:
            midProc.append(1)
            lenRecv.append(len(newProcessItems))
            for item in newProcessItems:
                procQueue.AddQueueItem(*item)
        else:
            midProc.append(0)
            lenRecv.append(0)

        botProc.append(qpa.ProcessArray(procQueue, nextItem.inTime))

    eventTimes.append(qpa._daqNode._absTimeNow-timeNow)
    hitsPerEvent.append(qpa._daqNode.daqHits)
    qpa._daqNode.daqHits = 0
    timeNow += deltaT
    tickNow += deltaTick
    # print("ran through this many items:", items)
    # print('topProc:', np.sum(topProc), len(topProc), topProc[:20])
    # print('midProc:  ', np.sum(midProc), len(midProc), midProc[:20])
    # print('botProc:  ', np.sum(botProc), len(botProc), botProc[:20])
    # print('lenRecv:  ', np.sum(lenRecv), len(lenRecv), lenRecv[:20])
    # print('sum:', items-np.sum(topProc+midProc+botProc))

In [3]:
qpa.Calibrate(5)
# qpa.PrintTimes()
print(qpa._daqNode.daqHits)

performing calibration..
calibration complete in 38 steps!
0
